# Handle the 'data' of the fishes
Wrestle with our data and Mikkis XLS sheet

In [292]:
import platform
import os
import glob
import pandas
import imageio
import numpy
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import seaborn
import dask
import dask_image.imread
from dask.distributed import Client, LocalCluster
from numcodecs import Blosc
import skimage
from tqdm import notebook

In [293]:
# Set dask temporary folder
# Do this before creating a client: https://stackoverflow.com/a/62804525/323100
import tempfile
if 'Linux' in platform.system():
    tmp = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD')
elif 'Darwin' in platform.system():
    tmp = tempfile.gettempdir()
else:
    if 'anaklin' in platform.node():
        tmp = os.path.join('F:\\')
    else:
        tmp = os.path.join('D:\\')
dask.config.set({'temporary_directory': os.path.join(tmp, 'tmp')})
print('Dask temporarry files go to %s' % dask.config.get('temporary_directory'))

Dask temporarry files go to D:\tmp


In [294]:
# Start cluster and client now, after setting tempdir
cluster = LocalCluster(n_workers=8)
client = Client(cluster)

C:\Users\haberthu\Miniconda3\lib\site-packages\distributed\node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 54785 instead
  warnings.warn(


In [295]:
print('You can seee what DASK is doing at "http://localhost:%s/status"' % client.scheduler_info()['services']['dashboard'])

You can seee what DASK is doing at "http://localhost:54785/status"


In [296]:
# # Ignore warnings in the notebook
# import warnings
# warnings.filterwarnings("ignore")

In [297]:
# Set up figure defaults
plt.rc('image', cmap='gray', interpolation='nearest')  # Display all images in b&w and with 'nearest' interpolation
plt.rcParams['figure.figsize'] = (16, 9)  # Size up figures a bit
plt.rcParams['figure.dpi'] = 200

In [298]:
# Setup scale bar defaults
plt.rcParams['scalebar.location'] = 'lower right'
plt.rcParams['scalebar.frameon'] = False
plt.rcParams['scalebar.color'] = 'white'

In [299]:
# Display all plots identically
lines = 3
# And then do something like
# plt.subplot(lines, numpy.ceil(len(Data) / float(lines)), c + 1)

In [300]:
# Different locations if running either on Linux or Windows
FastSSD = False
overthere = False # Load the data directly from the iee-research_storage drive
# to speed things up significantly
if 'Linux' in platform.system():
    if FastSSD:
        BasePath = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD')
    else:
        BasePath = os.path.join(os.sep, 'home', 'habi', '1272')
elif 'Darwin' in platform.system():
    FastSSD = False
    BasePath = os.path.join('/Users/habi/Dev/EAWAG/Data')
elif 'Windows' in platform.system():
    if FastSSD:
        BasePath = os.path.join('F:\\')
    else:
        if 'anaklin' in platform.node():
            BasePath = os.path.join('S:\\')
        else:
            BasePath = os.path.join('D:\\Results')
Root = os.path.join(BasePath, 'EAWAG')
if overthere:
        Root = os.path.join('I:\\microCTupload')
print('We are loading all the data from %s' % Root)

We are loading all the data from D:\Results\EAWAG


In [301]:
def get_pixelsize(logfile):
    """Get the pixel size from the scan log file"""
    pixelsize=None    
    with open(logfile, 'r') as f:
        for line in f:
            if 'Image Pixel' in line and 'Scaled' not in line:
                pixelsize = float(line.split('=')[1])
    return(pixelsize)

In [302]:
def get_projectionsize(logfile):
    """How big did we set the camera?"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Number Of Rows' in line:
                y = int(line.split('=')[1])
            if 'Number Of Columns' in line:
                x = int(line.split('=')[1])                
    return(x*y)

In [303]:
def get_filter(logfile):
    """Get the filter we used whole scanning from the scan log file"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Filter=' in line:
                whichfilter = line.split('=')[1].strip()
    return(whichfilter)

In [304]:
def get_exposuretime(logfile):
    """Get the exposure time size from the scan log file"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Exposure' in line:
                exposuretime = int(line.split('=')[1])
    return(exposuretime)

In [305]:
def get_ringartefact(logfile):
    """Get the ring artefact correction from the  scan log file"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Ring Artifact' in line:
                ringartefactcorrection = int(line.split('=')[1])
    return(ringartefactcorrection)

In [306]:
def get_reconstruction_grayvalue(logfile):
    grayvalue = None
    """How did we map the brightness of the reconstructions?"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Maximum for' in line:
                grayvalue = float(line.split('=')[1])
    return(grayvalue)

In [307]:
def get_beamhardening(logfile):
    """Get the beamhardening correction from the  scan log file"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Hardening' in line:
                beamhardeningcorrection = int(line.split('=')[1])
    return(beamhardeningcorrection)

In [308]:
def get_rotationstep(logfile):
    """Get the rotation step from the scan log file"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Rotation Step' in line:
                rotstep = float(line.split('=')[1])
    return(rotstep)

In [309]:
def get_frameaveraging(logfile):
    """Get the frame averaging from the scan log file"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Averaging' in line:
                avg = line.split('=')[1]
    return(avg)

In [310]:
def get_machine(logfile):
    """Get the machine we used to scan"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Scanner' in line:
                machine = line.split('=')[1].strip()
    return(machine)

In [311]:
def get_scantime(logfile):
    """How long did we scan?"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Scan duration' in line:
                time = line.split('=')[1].strip()
    return(pandas.to_timedelta(time))

In [312]:
def get_stacks(logfile):
    """How many stacks/connected scans did we make?"""
    stacks = 1
    with open(logfile, 'r') as f:
        for line in f:
            if 'conn' in line:
                stacks = int(line.split('=')[1])
    return(stacks)

In [313]:
def get_scandate(logfile, verbose=False):
    """When did we scan the fish?"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Study Date and Time' in line:
                if verbose:
                    print('Found "date" line: %s' % line.strip())
                datestring = line.split('=')[1].strip().replace('  ', ' ')
                if verbose:
                    print('The date string is: %s' % datestring)
                date = pandas.to_datetime(datestring , format='%d %b %Y %Hh:%Mm:%Ss')
                if verbose:
                    print('Parsed to: %s' % date)
                (date)
    return(date.isoformat())

In [314]:
def get_git_hash():
    '''
    Get the current git hash from the repository.
    Based on http://stackoverflow.com/a/949391/323100 and
    http://stackoverflow.com/a/18283905/323100
    '''
    from subprocess import Popen, PIPE
    import os
    gitprocess = Popen(['git',
                        '--git-dir',
                        os.path.join(os.getcwd(), '.git'),
                        'rev-parse',
                        '--short',
                        '--verify',
                        'HEAD'],
                       stdout=PIPE)
    (output, _) = gitprocess.communicate()
    return output.strip().decode("utf-8")

In [315]:
# # Make directory for output
# OutPutDir = os.path.join(os.getcwd(), 'Output', get_git_hash())
# print('We are saving all the output to %s' % OutPutDir)
# os.makedirs(OutPutDir, exist_ok=True)

In [316]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [317]:
# Get *all* log files
# Sort them by time, not name
Data['LogFile'] = [f for f in sorted(glob.glob(os.path.join(Root, '**', '*.log'), recursive=True), key=os.path.getmtime)]

In [318]:
# Get all folders
Data['Folder'] = [os.path.dirname(f) for f in Data['LogFile']]

In [319]:
# Check for samples which are not yet reconstructed
for c, row in Data.iterrows():
    # Iterate over every 'proj' folder
    if 'proj' in row.Folder:
        if not 'TScopy' in row.Folder and not 'PR' in row.Folder:
            # If there's nothing with 'rec*' on the same level, then tell us        
            if not glob.glob(row.Folder.replace('proj', 'rec')):
                print(glob.glob(row.Folder.replace('proj', 'rec')))
                print('- %s is missing matching reconstructions' % row.LogFile[len(Root)+1:])

[]
- 105005_104015\proj\105005_104015~00.log is missing matching reconstructions
[]
- 105005_104015\proj\105005_104015~01.log is missing matching reconstructions
[]
- 105005_104015\proj\105005_104015~02.log is missing matching reconstructions
[]
- 105005_104015\proj\105005_104015~03.log is missing matching reconstructions
[]
- 105005_104015\proj\105005_104015~04.log is missing matching reconstructions
[]
- 105005_104015\proj\105005_104015.log is missing matching reconstructions
[]
- 104671_156645\proj\104671_156645~00.log is missing matching reconstructions
[]
- 104671_156645\proj\104671_156645~01.log is missing matching reconstructions
[]
- 104671_156645\proj\104671_156645~02.log is missing matching reconstructions
[]
- 104671_156645\proj\104671_156645.log is missing matching reconstructions
[]
- MA41\proj\MA41.log is missing matching reconstructions
[]
- MA41\proj\MA41_rectmp.log is missing matching reconstructions
[]
- MA41\proj\MA41~00.log is missing matching reconstructions
[]
- M

In [320]:
# Get rid of all non-rec logfiles
for c, row in Data.iterrows():
    if 'rec' not in row.Folder:
        Data.drop([c], inplace=True)
    elif 'rectmp.log' in row.LogFile:
        Data.drop([c], inplace=True)
# Reset dataframe to something that we would get if we only would have loaded the 'rec' files
Data = Data.reset_index(drop=True)

In [321]:
# Generate us some meaningful colums
Data['Fish'] = [l[len(Root)+1:].split(os.sep)[0] for l in Data['LogFile']]
Data['Scan'] = ['_'.join(l[len(Root)+1:].split(os.sep)[1:-1]) for l in Data['LogFile']]

In [322]:
# Get the file names of the reconstructions
Data['Reconstructions'] = [sorted(glob.glob(os.path.join(f, '*rec0*.png'))) for f in Data['Folder']]
Data['Number of reconstructions'] = [len(r) for r in Data.Reconstructions]

In [324]:
# Get the file names of the reconstructions
for c, row in Data[Data['Number of reconstructions'] == 0].iterrows():
    print('%s/%s: Scan %s does not contain any reconstructions and '
          'will be removed in the next step' % (c+1, len(Data), os.path.join(row.Fish, row.Scan)))

1/210: Scan 103908\jaw_rec does not contain any reconstructions and will be removed in the next step
2/210: Scan Teeth\W_rec_al0.25 does not contain any reconstructions and will be removed in the next step
3/210: Scan Teeth\W_rec_nofilter does not contain any reconstructions and will be removed in the next step
4/210: Scan Teeth\P_rec_al0.25 does not contain any reconstructions and will be removed in the next step
5/210: Scan Teeth\P_rec_nofilter does not contain any reconstructions and will be removed in the next step
6/210: Scan 104016\pharynx_rec does not contain any reconstructions and will be removed in the next step
7/210: Scan 103375\rec_stuck does not contain any reconstructions and will be removed in the next step
8/210: Scan 103375\rec does not contain any reconstructions and will be removed in the next step
9/210: Scan NY75\rec does not contain any reconstructions and will be removed in the next step
10/210: Scan 161543\head_30um_rec does not contain any reconstructions and 

In [325]:
# Drop samples which have not been reconstructed yet
# Based on https://stackoverflow.com/a/13851602
# for c,row in Data.iterrows():
#     if not row['Number of reconstructions']:
#         print('%s contains no PNG files, we might be currently reconstructing it' % row.Folder)
print('We have %s folders in total' % (len(Data)))
Data = Data[Data['Number of reconstructions'] > 0]
Data.reset_index(drop=True, inplace=True)
print('We have %s folders *with* reconstructions in them' % (len(Data)))

We have 210 folders in total
We have 116 folders *with* reconstructions in them


In [326]:
# Get parameters to doublecheck from logfiles
Data['Voxelsize'] = [get_pixelsize(log) for log in Data['LogFile']]
Data['Filter'] = [get_filter(log) for log in Data['LogFile']]
Data['Exposuretime'] = [get_exposuretime(log) for log in Data['LogFile']]
Data['Scanner'] = [get_machine(log) for log in Data['LogFile']]
Data['Averaging'] = [get_frameaveraging(log) for log in Data['LogFile']]
Data['ProjectionSize'] = [get_projectionsize(log) for log in Data['LogFile']]
Data['RotationStep'] = [get_rotationstep(log) for log in Data['LogFile']]
Data['CameraWindow'] = [round((ps ** 0.5)/100)*100  for ps in Data['ProjectionSize']]
Data['Grayvalue'] = [get_reconstruction_grayvalue(log) for log in Data['LogFile']]
Data['RingartefactCorrection'] = [get_ringartefact(log) for log in Data['LogFile']]
Data['BeamHardeningCorrection'] = [get_beamhardening(log) for log in Data['LogFile']]
Data['Scan date'] = [get_scandate(log) for log in Data['LogFile']]
Data['Scan time'] = [get_scantime(log) for log in Data['LogFile']]
Data['Stacks'] = [get_stacks(log) for log in Data['LogFile']]

In [327]:
Data['Scan time total'] = [ st * stk  for st, stk in zip(Data['Scan time'], Data['Stacks'])]

In [328]:
# Generate a text file for each rec-folder, in which we can note what's going on with the fish
# Generate filename
for c,row in Data.iterrows():
    Data.at[c, 'commentsfile'] = os.path.join(os.path.dirname(row.Folder),
                                              row.Fish + '.' + row.Scan + '.md')
# Create actual file on disk
for c,row in Data.iterrows():
    # Only do this if the file does not already exist
    if not os.path.exists(row.commentsfile):
        with open(row.commentsfile, 'w', encoding='utf-8') as f:
            f.write('# Fish %s, Scan %s\n\n' % (row.Fish, row.Scan))
            f.write('This fish was scanned on %s on the %s, with a voxel size of %s μm.\n\n'
                    % (row['Scan date'], row.Scanner, numpy.round(row.Voxelsize, 2)))
            f.write('## Comments')

In [329]:
# # https://www.geeksforgeeks.org/iterating-over-rows-and-columns-in-pandas-dataframe/
# columns = list(Data)
# columns.remove('Folder') 
# columns.remove('Fish')
# columns.remove('LogFile')
# columns.remove('Reconstructions')
# columns.remove('Number of reconstructions')
# columns.remove('Grayvalue')
# columns.remove('Scan time')
# columns.remove('Scan time total')
# columns.remove('Scan date')
# print(columns)
# for col in columns:
#     print(col)
#     print(Data[col].unique())
#     print(80*'-')    

In [330]:
# # Check voxel sizes (*rounded* to two after-comma values)
# # If different, spit out which values
# roundto = 2
# if len(Data['Voxelsize'].round(roundto).unique()) > 1:
#     print('We scanned all datasets with %s different voxel sizes' % len(Data['Voxelsize'].round(roundto).unique()))
#     for vs in sorted(Data['Voxelsize'].round(roundto).unique()):
#         print('-', vs, 'um for ', end='')
#         for c, row in Data.iterrows():
#             if float(vs) == round(row['Voxelsize'], roundto):
#                 print(os.path.join(row['Fish'], row['Scan']), end=', ')
#         print('')
# else:
#     print('We scanned all datasets with equal voxel size, namely %s um.' % float(Data['Voxelsize'].round(roundto).unique()))

In [331]:
# if len(Data['Grayvalue'].unique()) > 1:
#     print('We reconstructed the datasets with different maximum gray values, namely')
#     for gv in Data['Grayvalue'].unique():
#         print(gv, 'for Samples ', end='')
#         for c, row in Data.iterrows():
#             if float(gv) == row['Grayvalue']:
#                 print(os.path.join(row['Fish'], row['Scan']), end=', ')
#         print('')
# else:
#     print('We reconstructed all datasets with equal maximum gray value, namely %s.' % Data['Grayvalue'].unique()[0])

In [332]:
# Data[['Fish', 'Scan',
#       'Voxelsize', 'Scanner',
#       'Scan date', 'CameraWindow', 'RotationStep', 'Averaging',
#       'Scan time', 'Stacks', 'Scan time total']]

In [333]:
# Get an overview over the total scan time
# Nice output based on https://stackoverflow.com/a/8907407/323100
total_seconds = int(Data['Scan time total'].sum().total_seconds())
hours, remainder = divmod(total_seconds,60*60)
minutes, seconds = divmod(remainder,60)
print('In total, we scanned for %s hours and %s minutes)' % (hours, minutes))
for machine in Data['Scanner'].unique():
    total_seconds = int(Data[Data['Scanner'] == machine]['Scan time total'].sum().total_seconds())
    hours, remainder = divmod(total_seconds,60*60)
    minutes, seconds = divmod(remainder,60)
    print('\t - Of these, we scanned %s hours and %s minutes on the %s,'
          'for %s scans' % (hours,
                            minutes,
                            machine,
                            len(Data[Data['Scanner'] == machine])))

In total, we scanned for 409 hours and 43 minutes)
	 - Of these, we scanned 276 hours and 9 minutes on the SkyScan2214,for 106 scans
	 - Of these, we scanned 133 hours and 33 minutes on the SkyScan1272,for 10 scans


In [334]:
Data[['Fish', 'Scan',
      'Voxelsize', 'Scanner',
      'Scan date', 'CameraWindow', 'RotationStep', 'Averaging', 'Scan time', 'Stacks' ]].to_excel('Details.xlsx')

In [335]:
Data[['Fish', 'Scan',
      'Voxelsize', 'Scanner',
      'Scan date', 'CameraWindow',
      'RotationStep', 'Averaging', 'Scan time', 'Stacks' ]].to_excel(os.path.join(Root,'Details.xlsx'))

In [336]:
# Read Mikkis datafile
MikkisFile = sorted(glob.glob(os.path.join(Root, 'X_ArchiveFiles', '*CTscanFishList.xlsx')))[0]
# Read excel file and use the first column as index
print('Reading in %s' % MikkisFile)
DataMikki = pandas.read_excel(MikkisFile)

Reading in D:\Results\EAWAG\X_ArchiveFiles\02.07.2021_CTscanFishList.xlsx


In [337]:
DataMikki.head()

,Fishec,FieldID,OtherID,ReplacementID,Length(cm),TemporaryJar,Genus,Species,Ecology,Scan date,HeadScan,OralJawScan,PharyngealJawScan,OperculumVisible,DataUploaded,QualityChecked,ScanComments,SpecimenReturned,Comments
0,103635,NaN,NaN,NaN,< 7,< 7cm,"""Astatotilapia""",nubila swamp blue,insectivore,2021-02-08T12:25:19,no 20um headscan,yes,yes,no 20um headscan,NaN,NaN,2-3 inner row of tricuspid teeth,NaN,NaN
1,103635,NaN,NaN,NaN,< 7,< 7cm,"""Astatotilapia""",nubila swamp blue,insectivore,2021-02-08T14:24:12,no 20um headscan,yes,yes,no 20um headscan,NaN,NaN,2-3 inner row of tricuspid teeth,NaN,NaN
2,104016,NaN,NaN,NaN,< 7,< 7cm,Enterochromis I,cinctus (St. E),detritivore,2021-02-04T11:21:23,no,yes,not complete,no,NaN,NaN,pharyngeal jaw not complete,NaN,NaN
3,104016,NaN,NaN,NaN,< 7,< 7cm,Enterochromis I,cinctus (St. E),detritivore,2021-02-04T13:30:11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14298,NaN,NaN,NaN,< 7,< 7cm,Incertae sedis,thick skin,insectivore,NaN,no,yes,NaN,no,NaN,NaN,bad segmentation quality,NaN,NaN


In [338]:
# Find the fish we look at and display all the info we know about it
# Set a substring you're looking for to the variable below
# In which jar can we find it?
fish = 'MA31'

In [339]:
# Do we have something from this fish on disk?
ondisk = glob.glob(os.path.join(Root, '*%s*' % fish))
if len(ondisk):
    for found in ondisk:
        print('*%s*: Found on disk in %s' % (fish, found))
        foundondisk = 1
else:
    print('*%s*: Nothing found in %s' % (fish, Root))
    foundondisk = 0

*MA31*: Found on disk in D:\Results\EAWAG\MA31


In [340]:
# Did we scan it already?
found = 0
for c, row in Data.iterrows():
    if fish in row.Fish:
        print('*%s*: Sample %s/%s was scanned on %s' % (fish, row['Fish'], row['Scan'], row['Scan date']))
        found = 1
if not found:
    if foundondisk:
        print('*%s*: We have a folder (%s) for this sample, but nothing in the dataframe, so it probably is all good' % (fish, ondisk[0]))
        print('Check the folder to be shure')
    else:
        print('*%s*: Nothing about this sample is found in our dataframe' % fish)

*MA31*: Sample MA31/rec_rescan was scanned on 2021-08-20T14:28:47
*MA31*: Sample MA31/rec_rescan_rereconstruct_OJ was scanned on 2021-08-20T14:28:47
*MA31*: Sample MA31/head_rec was scanned on 2021-10-19T11:15:30


In [341]:
# Can we find it in FullHeadList.txt?
def findinFullHeadList(sample):
    ''' Look for the sample in the FullHeadList.txt file'''
    fullheadlist = glob.glob(os.path.join(Root, 'FullHeadList.*'))[0]    
    found = 0
    with open(fullheadlist, 'r') as f:
        for line in f:
            if str(sample) in line:
                print(line.strip())
                found = 1
    if not found:
        return('*%s*: Nothing found in %s' % (sample, fullheadlist))
    else:
        return(None)
findinFullHeadList(fish)

'*MA31*: Nothing found in D:\\Results\\EAWAG\\FullHeadList.txt'

In [342]:
# Do we need to rescan this fish
# Find all relevant comment files
commentfiles = glob.glob(os.path.join(Root, '*%s*' % fish, '**', '*.md'), recursive=True)

print('We found these comment files in our dataframe')
for c, row in Data.iterrows():
    if fish in row.Fish:
        print('\t-', row.commentsfile)
        found = 1

print('We found these comment files on disk')
if len(commentfiles):
    for commentfile in commentfiles:
        with open(commentfile, 'r', encoding='utf-8') as file:
            for line in file:
                print(line.strip())
                if 'rescan' in line:
                    print('BEEEEP!')      
else:
    print('No file for potential comments found, sample %s can be returned to its jar')
    print('See below')

We found these comment files in our dataframe
	- D:\Results\EAWAG\MA31\MA31.rec_rescan.md
	- D:\Results\EAWAG\MA31\MA31.rec_rescan_rereconstruct_OJ.md
	- D:\Results\EAWAG\MA31\head\MA31.head_rec.md
We found these comment files on disk
# Fish MA31, Scan rec_rescan
BEEEEP!

This fish was scanned on 2021-08-20T14:28:47 on the SkyScan2214, with a voxel size of 9.3 μm.

## Comments

Side note - previous scan was done at 8.59um.
The rescan shows artifacts on oral jaw teeth which stick them together - QualityCheck png.
BEEEEP!
# Fish MA31, Scan rec_rescan_rereconstruct_OJ
BEEEEP!

This fish was scanned on 2021-08-20T14:28:47 on the SkyScan2214, with a voxel size of 9.3 μm.

## Comments
# Fish MA31, Scan head_rec

This fish was scanned on 2021-10-19T11:15:30 on the SkyScan2214, with a voxel size of 17.0 μm.

## Comments


In [373]:
# In which jar should it be/go?
foundfishes = 0
for d, row in DataMikki.iterrows():
    if (str(fish).lower() in str(row.Fishec).lower()) or \
    (str(fish).lower() in str(row.FieldID).lower()) or \
    (str(fish).lower() in str(row.OtherID).lower()) or \
    (str(fish).lower() in str(row.ReplacementID).lower()):
        foundfishes = (row.Fishec, row.FieldID, row.OtherID, row.ReplacementID)
        # remove nan from the list of hits
        foundfishes = [str(x).lower() for x in foundfishes if pandas.isnull(x) == False]
        print('*%s*: The fish ' % fish, end='')        
        if len(foundfishes) > 1:
            for found in foundfishes:
                print(found.upper(), end='/')
        else:
            print(foundfishes[0].upper(), end='')
        print(' should now go in jar "length=%s cm" (%s))' % (row['Length(cm)'],
                                                              row['TemporaryJar']))
if not foundfishes:
    print('*%s*: Nothing found in %s' % (fish, MikkisFile))

*105105*: The fish 105105 should now go in jar "length=15 cm" (Mark1))


60 of the fishes need complete head scans.
Let's try to go through Mikkis/Kassandras list and see how far we progressed through that list.

In [374]:
# Read in full head list, go through all the scans we alredy did and see what needs to be done
fullheadlist = glob.glob(os.path.join(Root, '*Head*.txt'))[0]
HeadsToBeScanned = []
with open(fullheadlist, 'r', encoding='utf-8') as file:
    headdone = False
    for ln, line in enumerate(file):
        if line.strip():  #skip empty lines
            # The firs 'item' on the line should be the fish ID
            fish = line.split()[0].replace(',','').replace('asdf','').upper()
            # Let's ignore some lines which don't start with a fish ID
            # The set-join here removes duplicate characters from the string (e.g. =====, !! and ::)
            if len(''.join(set(fish))) > 2:
                for c, row in Data[Data.Fish == fish].iterrows():
                    if 'head' in row.Scan:
                        # print('\t%s has a head-scan' % row.Fish)
                        # print('%s has a head-scan on disk, and is found on line %s of the full head list' % (fish, ln + 1 ))
                        headdone = True
                    else:
                        headdone = False
                # At this point we have either found the fish in the list or 'headdone' is false
                if not headdone:
                    print('%s is missing a head-scan on disk, but is found on line %s of the full head list' % (fish, ln + 1 ))
                    HeadsToBeScanned.append(fish)             

104061 is missing a head-scan on disk, but is found on line 32 of the full head list
13115 is missing a head-scan on disk, but is found on line 42 of the full head list
10448 is missing a head-scan on disk, but is found on line 48 of the full head list
103761 is missing a head-scan on disk, but is found on line 72 of the full head list
11447 is missing a head-scan on disk, but is found on line 73 of the full head list
105105 is missing a head-scan on disk, but is found on line 74 of the full head list


In [370]:
# Fish 10448 can be ignored because we did another scan after the head-scan, so we reset "headdone" in the loop above
# We could probably do it in a more clever way, but already spent too much time on this part :)
try:
    HeadsToBeScanned.remove('10448')
except ValueError:
    # Nothing to see here, pass along
    pass

In [372]:
for fish in HeadsToBeScanned:
    # In which jar should we look for the fishes we still need to scan the head of?
    foundfishes = 0
    for d, row in DataMikki.iterrows():
        if (str(fish).lower() in str(row.Fishec).lower()) or \
        (str(fish).lower() in str(row.FieldID).lower()) or \
        (str(fish).lower() in str(row.OtherID).lower()) or \
        (str(fish).lower() in str(row.ReplacementID).lower()):
            foundfishes = (row.Fishec, row.FieldID, row.OtherID, row.ReplacementID)
            # remove nan from the list of hits
            foundfishes = [str(x).lower() for x in foundfishes if pandas.isnull(x) == False]
            print('*%s*: A fish called ' % fish, end='')        
            if len(foundfishes) > 1:
                for found in foundfishes:
                    print(found.upper(), end='/')
            else:
                print(foundfishes[0].upper(), end='')
            print(' should be found in jar "length=%s cm" (%s))' % (row['Length(cm)'],
                                                                    row['TemporaryJar']))
    if not foundfishes:
        print('*%s*: Nothing found in %s' % (fish, MikkisFile))

*104061*: A fish called 13405/104061/ should be found in jar "length=14.5 cm" (Mark5))
*13115*: A fish called 11317/13115/ should be found in jar "length=18 cm" (Mark4))
*103761*: A fish called 104626/103761/ should be found in jar "length=8 cm" (Mark3))
*11447*: A fish called 11447 should be found in jar "length=7.9 cm" (Mark2))
*105105*: A fish called 105105 should be found in jar "length=15 cm" (Mark1))
